In [1]:
!pip install opencv-python


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: C:\Users\Kevin\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip


In [2]:
import threading
import cv2 as cv
import numpy as np

face_cascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")

In [3]:
# võetud https://stackoverflow.com/a/44659589
def image_resize(image, width = None, height = None, inter = cv.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv.resize(image, dim, interpolation = inter)
    # return the resized image
    return resized

In [38]:
def overlayPng(background, overlay, x, y, w, h):
    # kisume overlay pildist alpha kanali välja
    alpha = overlay[:,:,3]
    alpha = cv.merge([alpha,alpha,alpha])

    background_cropped = background[y:y + h, x:x + w]

    # kisume overlay pildist BGR kanalid välja
    bgr = overlay[:,:,0:3]

    # kui alpha on 0, vali taust, kui mitte, vali meie overlay filter
    background[y:y + h, x:x + w] = np.where(alpha==0, background_cropped, bgr)
    return background

In [35]:
head_keywords = ["fedora"]

def headFilter(img, x, y, w, filter_path):
    # loeme sisse overlay pildi koos alpha kanaliga
    filter = cv.imread(f"filters/{filter_path}.png", cv.IMREAD_UNCHANGED)
    filter = image_resize(filter, w)
    y_offset = y + int(filter.shape[0] * 0.2) - filter.shape[0]
    return overlayPng(img, filter, x, y_offset, w, filter.shape[0])

In [5]:
def videoWorker():
    video = cv.VideoCapture(0)
    global input
    selected_filter = "fedora"

    while True:
        result, img = video.read()

        # Flipib pildi ümber, et toimuks arusaadav peegeldus
        img = cv.flip(img, 1)

        # Näotuvastus
        gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_img)
        for (x, y, w, h) in faces:
            # Joonistab nägude ümber ristküliku
            cv.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
            if selected_filter:
                img = headFilter(img, x, y, w, selected_filter)

        cv.imshow("Camera", img)

        k = cv.waitKey(30) & 0xFF
        if k == 27: # ehk ESC nupp
            cv.destroyAllWindows()
            break

        if input == "exit":
            cv.destroyAllWindows()
            break

    video.release()
    cv.destroyAllWindows()

def inputWorker():
    global input
    global selected_filter

    while True:
        input = input()
        if input == "exit":
            break
        index = head_keywords.index(input)
        if index != -1:
            selected_filter = head_keywords[index]

# threading.Thread(target=videoWorker).start()
# threading.Thread(target=inputWorker).start()

In [ ]:
videoWorker()